In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.5 MB/s eta 0:00:00


In [11]:
from ultralytics import YOLO
import os, zipfile, shutil
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display

In [12]:
# Extract Your Dataset

zip_path = "/content/My First Project.v3i.yolov8.zip"   # update if path differs
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset Extracted Successfully!")
print(os.listdir(extract_path))

✅ Dataset Extracted Successfully!
['train', 'data.yaml', 'README.dataset.txt', 'valid', 'test', 'README.roboflow.txt']


In [13]:
# Check Dataset Structure

data_yaml_path = os.path.join(extract_path, "data.yaml")
with open(data_yaml_path, 'r') as f:
    print(f.read())

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['obeject_detection']

roboflow:
  workspace: edison-p91x6
  project: my-first-project-ibuft
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/edison-p91x6/my-first-project-ibuft/dataset/3


In [14]:
# Load & Train YOLOv8 Model

# You can choose: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium)
model = YOLO('yolov8n.pt')

# Train the model
results = model.train(
    data=data_yaml_path,        # YAML file path
    epochs=50,                  # increase if needed
    imgsz=640,                  # image size
    batch=16,
    name="traffic_detector_yolov8",
    device=0                    # use GPU if available
)
# Evaluate Model Performance
metrics = model.val()
print("📊 Evaluation metrics:", metrics)

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic_detector_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

In [17]:
# Export Model to TensorFlow Format

export_path = model.export(format="saved_model")  # TensorFlow SavedModel
print("✅ Model exported to TensorFlow format at:", export_path)

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/content/runs/detect/traffic_detector_yolov8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 16.1s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.72...
ONNX: export success ✅ 1.6s, saved as '/content/runs/detect/traffic_detector_yolov8/weights/best

In [18]:
# Load and Test the TensorFlow Model (Optional)

import tensorflow as tf

tf_model = tf.saved_model.load(export_path)
print("TensorFlow Model Loaded ✅")

# (Optional) list TensorFlow model signatures
for f in tf_model.signatures:
    print(f)

TensorFlow Model Loaded ✅
serving_default


In [19]:
# Visualize Training Results

results_dir = "runs/detect/traffic_detector_yolov8"
if os.path.exists(results_dir):
    print("Training results saved in:", results_dir)
else:
    print("Check 'runs/' folder for outputs")

Training results saved in: runs/detect/traffic_detector_yolov8


In [20]:
# ✅ Done! You can now deploy this model.

print("🎉 YOLOv8 Object Detection Training Completed Successfully!")

🎉 YOLOv8 Object Detection Training Completed Successfully!
